In [6]:
from haystack import Pipeline
import os

os.environ["OPENAI_API_KEY"] = "<your_openai_api_key>"

In [9]:
from haystack.components.generators.openai import OpenAIGenerator
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.retrievers.in_memory.embedding_retriever import InMemoryEmbeddingRetriever
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders.openai_text_embedder import OpenAITextEmbedder
from haystack.components.rankers.lost_in_the_middle import LostInTheMiddleRanker

pipeline = Pipeline()
pipeline.add_component("embedder", OpenAITextEmbedder(model="text-embedding-ada-002"))
pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store=InMemoryDocumentStore()))
pipeline.add_component("ranker", LostInTheMiddleRanker())
pipeline.add_component("prompt_builder", PromptBuilder(template="""

        Given these documents, answer the question.

        Your answer is used within a pipeline and must follow the following format:
        "The answer is "valid"" or "The answer is "invalid"" with quotation marks. Also allowed are "yes" or "no", "true" or "false", "1" or "0".
        After stating your answer, explain your answer in detail.

        Example:
        Question: London is the capital of France.
        Answer: "false", because the capital of France is Paris, not London.

        Documents:

        {% for doc in documents %}\

        {{ doc.content }}

        {% endfor %}        

        Question: {{query}}


        Answer:

        """))
pipeline.add_component("generator", OpenAIGenerator())

pipeline.connect("embedder", "retriever.query_embedding")
pipeline.connect("retriever", "ranker")
pipeline.connect("ranker", "prompt_builder")
pipeline.connect("prompt_builder", "generator")

pipeline.dump(open("pipeline.yaml", "w"))







